__Abstract__
 - Goal 
     - Apply importance sampling (IS) to Knock-In option.
 - Ref
     - Ex4.6.4 of [Gla03]

__Problem__

- Asset follows $GBM(S_0, r, \sigma^2)$;
- Knock-in payoff is given as
$$F(S) = I(S_T > K) \cdot I\Big(\min_{1\le k \le m} S(t_k) < H\Big).$$
- Find price $$e^{-rT}\mathbb E [F(S)]$$

__Anal__

__Parameters__

- $r = 5\%, \sigma = 15\%, S(0) = 95$
- $T = 0.25, m = 50, H = 85, K = 96.$
- $k = 100$ many MC computation; 
- $n = 1000$ paths in each MC;
- $m = 50$ time steps for each path

__Algo__ 

In [1]:
%run is_bsm_knock_in.py

MC computation value: 

Mean is 0.5534271442189959
MSE is 0.001559407163387847


In [2]:
%cat is_bsm_knock_in.py

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Nov 22 17:00:26 2018

@author: songqsh
"""

import numpy as np
from exact_simulation_v01 import bm_1d

r = .05; sigma = 0.15; s0 = 95
T = 0.25; H = 85; K = 96
#k = 1000; n = 1000; m = 50
k = 100; n = 1000; m = 50; dt = T/m

b = - np.log(H/s0); c = np.log(K/s0)
slope = (c+2*b)/T
r1 = .5*sigma**2 - slope
r2 = .5*sigma**2 + slope
th1 = (r1 - r)/sigma
th2 = (r2 -r)/sigma

val = np.zeros(k)
for ii in range(k):
    L = np.zeros(m)
    adj_payoff = np.zeros(n)
    
    for i in range(n):
        w_hat = bm_1d(T,m)
        flag = 0
        tau = T + 1
        w_hat_tau = 0
        
        #generate a path for L
        for j in range(m):
            if flag == 0:
                L[j] = sigma*w_hat[j] - slope*(j+1)*dt
                if L[j] < - b:
                    tau = (j+1)*dt
                    w_hat_tau = w_hat[j]
                    flag = 1
            else:
                L[j] = sigma*w_hat[j] -slope*tau + slope*((j